In [1]:
import xmltodict, json, feedparser
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time, re
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime

In [163]:
df = pd.read_json('calendarDB.json', orient='index')
df

,파크시스템스,파크시스템,파크시스,삼성전자,현대차,기아
2022-08-24,"[기업설명회(IR)개최, 반기실적 및 경영현황 업데이트]","[기업설명회(IR)개, 반기실적 및 경영현황 업데이트]","[기업설명회(IR), 반기실적 및 경영현황 업데이트]",NaN,NaN,NaN
2022-08-25,NaN,NaN,NaN,"[기업설명회(IR)개최, 반기실적 및 경영현황 업데이트]","[기업설명회(IR)개, 반기실적 및 경영현황 업데이트]","[기업설명회(IR), 반기실적 및 경영현황 업데이트]"


In [2]:
def makeCalendar():
  
  while True:
    
    class BreakIt(Exception): pass

    rss_info = feedparser.parse('http://dart.fss.or.kr/api/todayRSS.xml')
    disclosure_list = rss_info['entries']

    for disclosure in disclosure_list:
      companyName = disclosure['author']
      link = disclosure['link']
      title = disclosure['title']


      if ('기업설명회(IR)' in title) and not('안내공시' in title):
        
          print(title)
          print(datetime.now())
          for index, row in table_ifrm[0].iterrows():
            if '목적' in index:
              text = row[2]
          for index, row in table_ifrm[0].iterrows():
            if '일시' in index:
              if '20' in row[2]:
                dateNew = row[2]
        
      elif '기업설명회(IR)개최(안내공시)' in title:

        print(title)
        print(datetime.now())
        text = table_ifrm[0].loc['3. 개최목적', 2]
        dateNew = table_ifrm[0].loc['1. 일시 및 장소', 2][0]

      else:
        raise BreakIt


      try:
        
        option = Options()
        option.add_argument('headless')
        driver = webdriver.Chrome(options=option)
        driver.get(link)
        # time.sleep(2)
        html_source = driver.page_source

        rcpNo = re.findall(r"openPdfDownload\('(.*?)', '", html_source)
        dcmNo = re.findall(r"', '(.*?)'\);", html_source)

        # url이 있어야 read_html을 통해 table 가져올 수 있다
        frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[2][0:7] + '&eleId=' + '0' + '&offset=' + '0' + '&length=' + '0' + '&dtd=HTML'

        table_ifrm = pd.read_html(frame_url, index_col=0)#, header=None)

        
        # if ('기업설명회(IR)' in title) and not('안내공시' in title):
        
        #   print(title)
        #   print(datetime.now())
        #   for index, row in table_ifrm[0].iterrows():
        #     if '목적' in index:
        #       text = row[2]
        #   for index, row in table_ifrm[0].iterrows():
        #     if '일시' in index:
        #       if '20' in row[2]:
        #         dateNew = row[2]
        
        # if '기업설명회(IR)개최(안내공시)' in title:

        #   print(title)
        #   print(datetime.now())
        #   text = table_ifrm[0].loc['3. 개최목적', 2]
        #   dateNew = table_ifrm[0].loc['1. 일시 및 장소', 2][0]

        
        with open('calendarDB.json', encoding='utf-8') as db:
          res = json.load(db)

        seen = set()
        for date in res.keys():
          for company in res[date].keys():
            seen.add((date, company, res[date][company]['내용'], res[date][company]['링크']))

        # seen = {(date, company, res[date][company]['내용'], res[date][company]['링크']) for date in res.keys() for company in res[date].keys()} # 중첩된 집합 표기법?

        def add_entry(res, date, companyName, text, link):

            # check if in seen set
            if (date, companyName, text, link) in seen:
                return res

            # add to seen set
            seen.add(tuple([date, companyName, text, link]))
            print(seen)

            # append to results list
            try:
              res[date][companyName] = {'내용': text, '링크': link}
            except:
              res[date] = {companyName: {'내용': text, '링크': link}} # 해당날짜의 첫 번째 데이터

            return res

        args = [dateNew, companyName, text, link]
        print(args)

        res = add_entry(res, *args)

        with open('calendarDB.json', 'w', encoding='utf-8') as db:
          json.dump(res, db, ensure_ascii=False)

      except:
        pass
    print(datetime.now())
    time.sleep(600)


for i in range(1, 100):
  try:
    makeCalendar()
  except Exception as e:
    print(e)
    print('restarting')
    continue
  else:
    break

2022-09-08 12:24:53.879176
2022-09-08 12:43:07.795983
2022-09-08 12:58:38.157597
2022-09-08 13:14:58.661923
2022-09-08 13:30:36.904846
(유가)현대오토에버 - 기업설명회(IR)개최(안내공시)
2022-09-08 13:40:53.959282
['2022-09-15', '현대오토에버', "29th Annual CITIC CLSA Flagship Investors' Forum 2022 참가", 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220908800197']
{('2022-09-15', '현대오토에버', "29th Annual CITIC CLSA Flagship Investors' Forum 2022 참가", 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220908800197'), ('2022-09-14', '박셀바이오', '- 기업가치 및 투자가치 제고', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900285'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900153'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900213'), ('2022-09-07', '삼기', '회사에 대한 정보를 모든 투자자들이 쉽게 공유할 수 있도록 함으로써 공정한 정보 제공', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900169'), ('2022-09-05', '석경에이티', '2022년 반기 경영실적 발표 및 투자자

In [231]:
with open('calendarDB.json', encoding='utf-8') as db:
    res = json.load(db)


seen = ((date, res[date]['기업명'], res[date]['내용']) for date in res.keys())
# seen = (date for date in list(res.keys()))
set(seen)

{('2022-08-17', '3241', 'test3'), ('2022-08-24', '파크시스', '반기실적 및 경영현황 업데이트')}

In [13]:
from collections import defaultdict

d1 = defaultdict(list)

l = [[1, 'A'], [1, 'B'], [2, 'C']]

for k, v in l:
    d1[k].append(v)

d1.items()

# d = dict((k, tuple(v)) for k, v in d1.items())

# d

dict_items([(1, ['A', 'B']), (2, ['C'])])

In [39]:
seen = {(date, list(company for company in res[date].keys())) for date in res.keys()}
seen

TypeError: unhashable type: 'list'

In [34]:
res

{'2022-09-06': {'CJ ENM': {'내용': 'KB Korea Conference 2022'}}}

In [38]:
(company for company in res['2022-09-06'].keys())

<generator object <genexpr> at 0x000001362BB23140>

In [57]:
seen = set()
for date in res.keys():
  print(date)
  for company in res[date].keys():
    print(company)
    print(res[date][company]['내용'])
    seen.add((date, company, res[date][company]['내용']))

seen

2022-09-06
CJ ENM
KB Korea Conference 2022
2022-09-07
DJ ENM
NH Korea Conference 2022


{('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022'),
 ('2022-09-07', 'DJ ENM', 'NH Korea Conference 2022')}

In [91]:
res = {}
seen = set()

In [92]:


def add_entry(res, date, companyName, text):

  # check if in seen set
  if (date, companyName, text) in seen:
    return res

  # add to seen set
  seen.add(tuple([date, companyName, text]))
  print(seen)

  # append to results list
  res[date][companyName] = {'내용': text}
  # res[date] = {companyName: {'내용': text}}
  # res.append({date:{'기업명': companyName, '내용': text}})

  return res

args = ['2022-09-03', '기업명', '내용']
add_entry(res, *args)
print(res)

with open('calendarDB.json', 'w', encoding='utf-8') as db:
  json.dump(res, db, ensure_ascii=False)
db.close()

{('2022-09-03', '기업명', '내용')}


KeyError: '2022-09-03'

In [13]:
for i, key in enumerate(res.keys()):
  print(i, key, res[key])
  for j, key2 in enumerate(res[key]):
    print(j, key2)


0 2022-09-06 {'파라다이스': {'내용': '국내 기관투자자와의 커뮤니케이션'}, 'CJ ENM': {'내용': 'KB Korea Conference 2022'}}
0 파라다이스
1 CJ ENM


In [ ]:
companyName = '카카오'
title = '기업설명회(IR)'
link = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220907800492'
# html = urlopen('https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220823900266')
# bsObj = BeautifulSoup(html, 'html.parser')

# driver = webdriver.Chrome(ChromeDriverManager().install())
option = Options()
option.add_argument('headless')
driver = webdriver.Chrome(options=option)
driver.get(link)
# time.sleep(2)
html_source = driver.page_source


rcpNo = re.findall(r"openPdfDownload\('(.*?)', '", html_source)
dcmNo = re.findall(r"', '(.*?)'\);", html_source)

# url이 있어야 read_html을 통해 table 가져올 수 있다
frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[2][0:7] + '&eleId=' + '0' + '&offset=' + '0' + '&length=' + '0' + '&dtd=HTML'

table_ifrm = pd.read_html(frame_url, index_col=0)#, header=None)

# table_ifrm[0]
# date = defaultdict(dict)
# event = defaultdict(dict)

for index, row in table_ifrm[0].iterrows():
  if '목적' in index:
    text = row[2]
for index, row in table_ifrm[0].iterrows():
  if '일시' in index:
    if '20' in row[2]:
      dateNew = row[2]
    

with open('calendarDB.json', encoding='utf-8') as db:
  res = json.load(db)
# print(res)

seen = set()
# for date in res.keys():
#   for company in res[date].keys():
#     seen.add((date, company, res[date][company]['내용'], res[date][company]['링크']))
# print(seen)

seen = {(date, company, res[date][company]['내용'], res[date][company]['링크']) for date in res.keys() for company in res[date].keys()} # 중첩된 집합 표기법?

# seen = {(date, (company for company in res[date].keys()), res[date][company]['내용']) for date in res.keys()}
# seen = set(seen)


def add_entry(res, date, companyName, text, link):

    # check if in seen set
    if (date, companyName, text, link) in seen:
        return res

    # add to seen set
    seen.add(tuple([date, companyName, text, link]))
    print(seen)

    # append to results list
    try:
      res[date][companyName] = {'내용': text, '링크': link}
    except:
      res[date] = {companyName: {'내용': text, '링크': link}} # 해당날짜의 첫 번째 데이터
    # res.append({date:{'기업명': companyName, '내용': text}})

    return res

args = [dateNew, companyName, text, link]
print(args)

res = add_entry(res, *args)  # add entry - SUCCESS
# print(res)
res = add_entry(res, *args)  # try to add again - FAIL
# print(res)

# args2 = ['2022-08-17', 'LG에너지솔루션', 'test6']
# print(res)
# res = add_entry(res, *args2)  # add another - SUCCESS
# print(res)


with open('calendarDB.json', 'w', encoding='utf-8') as db:
  json.dump(res, db, ensure_ascii=False)
# db.close()  # with 블럭 끝나면 자동으로 닫아준다

# date['행사일']
# for key,val in event.items():
#     if key in d:
#         d[key] = [d[key],val]

# event[1].append
# table_ifrm[0].loc['1.일시', '2']

seen

In [24]:
companyName = '카카오'
title = '기업설명회(IR)'
link = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220907800492'

option = Options()
option.add_argument('headless')
driver = webdriver.Chrome(options=option)
driver.get(link)
# time.sleep(2)
html_source = driver.page_source

rcpNo = re.findall(r"openPdfDownload\('(.*?)', '", html_source)
dcmNo = re.findall(r"', '(.*?)'\);", html_source)

# url이 있어야 read_html을 통해 table 가져올 수 있다
frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[2][0:7] + '&eleId=' + '0' + '&offset=' + '0' + '&length=' + '0' + '&dtd=HTML'

table_ifrm = pd.read_html(frame_url, index_col=0)#, header=None)

# for index, row in table_ifrm[0].iterrows():
#   if '목적' in index:
#     text = row[2]
# for index, row in table_ifrm[0].iterrows():
#   if '일시' in index:
#     if '20' in row[2]:
#       dateNew = row[2]

text = table_ifrm[0].loc['3. 개최목적', 2]
dateNew = table_ifrm[0].loc['1. 일시 및 장소', 2][0]
    

with open('calendarDB.json', encoding='utf-8') as db:
  res = json.load(db)
# print(res)

seen = set()
for date in res.keys():
  for company in res[date].keys():
    seen.add((date, company, res[date][company]['내용'], res[date][company]['링크']))
print(seen)
# seen = {(date, company, res[date][company]['내용'], res[date][company]['링크']) for date in res.keys() for company in res[date].keys()} # 중첩된 집합 표기법?


def add_entry(res, date, companyName, text, link):

    # check if in seen set
    if (date, companyName, text, link) in seen:
        return res

    # add to seen set
    seen.add(tuple([date, companyName, text, link]))
    print(seen)

    # append to results list
    try:
      res[date][companyName] = {'내용': text, '링크': link}
    except:
      res[date] = {companyName: {'내용': text, '링크': link}} # 해당날짜의 첫 번째 데이터

    return res

args = [dateNew, companyName, text, link]
print(args)

res = add_entry(res, *args)  # add entry - SUCCESS

with open('calendarDB.json', 'w', encoding='utf-8') as db:
  json.dump(res, db, ensure_ascii=False)

print(seen)

{('2022-09-07', '삼기', '회사에 대한 정보를 모든 투자자들이 쉽게 공유할 수 있도록 함으로써 공정한 정보 제공', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900169'), ('2022-09-14', '박셀바이오', '- 기업가치 및 투자가치 제고', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900285'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900213'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900153'), ('2022-09-05', '석경에이티', '2022년 반기 경영실적 발표 및 투자자의 이해증진', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900160')}
['2022-09-15', '카카오', '2022년 9월 국내외 NDR 및 컨퍼런스', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220907800492']
{('2022-09-07', '삼기', '회사에 대한 정보를 모든 투자자들이 쉽게 공유할 수 있도록 함으로써 공정한 정보 제공', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900169'), ('2022-09-14', '박셀바이오', '- 기업가치 및 투자가치 제고', 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900285'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'https:

In [23]:
table_ifrm[0].loc['1. 일시 및 장소', 2][0]
table_ifrm[0].loc['3. 개최목적', 2]
# for index, row in table_ifrm[0].iterrows():
#   if '목적' in index:
#     text = row[2]
# for index, row in table_ifrm[0].iterrows():
#   if '일시' in index:
#     if '20' in row[2]:
#       dateNew = row[2]

'2022년 9월 국내외 NDR 및 컨퍼런스'

In [21]:
with open('calendarDB.json', encoding='utf-8') as db:
  res = json.load(db)

res.keys()

dict_keys(['2022-09-06', '2022-09-14', '2022-09-07', '2022-09-05'])

In [175]:
event

defaultdict(dict, {'파크시스': ['기업설명회(IR)', '반기실적 및 경영현황 업데이트']})

In [172]:
event2 = defaultdict(dict)
event2

for key, val in event2.items():
    if key in d:
        d[key] = [d[key],val]

defaultdict(dict, {})

In [14]:
date

'2022-09-06'

In [166]:
table_ifrm[0]

,1,2,3,4,5
0,,,,,
1. 일시,1. 일시,행사일,행사일,시간(현지시간),시간(현지시간)
1. 일시,1. 일시,시작일,종료일,시작시간,종료시간
1. 일시,1. 일시,2022-08-24,2022-08-25,09:00,16:00
2. 장소,2. 장소,-,-,-,-
3. 대상자,3. 대상자,국내 기관투자자,국내 기관투자자,국내 기관투자자,국내 기관투자자
4. 실시목적,4. 실시목적,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트
5. 실시방법,5. 실시방법,컨퍼런스콜 형식의 NDR,컨퍼런스콜 형식의 NDR,컨퍼런스콜 형식의 NDR,컨퍼런스콜 형식의 NDR
6. 주요내용,6. 주요내용,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트
7. 후원기관,7. 후원기관,BNK증권,BNK증권,BNK증권,BNK증권


In [165]:
date

defaultdict(dict,
            {'2022-08-24': defaultdict(dict,
                         {'파크시스템스': ['기업설명회(IR)개최', '반기실적 및 경영현황 업데이트'],
                          '파크시스템': ['기업설명회(IR)개', '반기실적 및 경영현황 업데이트'],
                          '파크시스': ['기업설명회(IR)', '반기실적 및 경영현황 업데이트']})})

In [26]:
rss_info = feedparser.parse('http://dart.fss.or.kr/api/todayRSS.xml')
disclosureList = rss_info['entries']
disclosureList[0]

{'title': '(코스닥)에스트래픽 - 주주명부폐쇄기간또는기준일설정',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://dart.fss.or.kr/api/todayRSS.xml',
  'value': '(코스닥)에스트래픽 - 주주명부폐쇄기간또는기준일설정'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220823900466'}],
 'link': 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220823900466',
 'tags': [{'term': '코스닥', 'scheme': None, 'label': None}],
 'published': 'Tue, 23 Aug 2022 09:16:00 GMT',
 'published_parsed': time.struct_time(tm_year=2022, tm_mon=8, tm_mday=23, tm_hour=9, tm_min=16, tm_sec=0, tm_wday=1, tm_yday=235, tm_isdst=0),
 'id': 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220823900466',
 'guidislink': False,
 'authors': [{'name': '에스트래픽'}],
 'author': '에스트래픽',
 'author_detail': {'name': '에스트래픽'},
 'updated': '2022-08-23T09:16:00Z',
 'updated_parsed': time.struct_time(tm_year=2022, tm_mon=8, tm_mday=23, tm_hour=9, tm_min=16, tm_sec=0, tm_wday=1, tm_yday=235, tm_isdst=0)}

In [11]:
with open('http://dart.fss.or.kr/api/todayRSS.xml') as dx:
  doc = xmltodict.parse(dx.read())

doc

OSError: [Errno 22] Invalid argument: 'https://dart.fss.or.kr/api/todayRSS.xml'

In [ ]:
print('hello again')

hello again
